##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Canalizaciones simples de TFX para Vertex Pipelines


<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/gcp/vertex_pipelines_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tfx/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a></td>
<td><a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tfx/tutorials/tfx/gcp/vertex_pipelines_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Ejecutar en Google Cloud Vertex AI Workbench</a></td>
</table></div>


En este tutorial basado en un bloc de notas, crearemos una canalización simple de TFX y la ejecutaremos con ayuda de Google Cloud Vertex Pipelines. Este bloc de notas se basa en la canalización de TFX que compilamos en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple). Si no está familiarizado con TFX y aún no ha leído ese tutorial, debe leerlo antes de continuar con este bloc de notas.

Google Cloud Vertex Pipelines lo ayuda a automatizar, monitorear y gobernar sus sistemas de aprendizaje automático (ML) al orquestar su flujo de trabajo de ML sin servidor. Puede definir sus canalizaciones de ML al usar Python con TFX y luego ejecutarlas en Google Cloud. Consulte la [Introducción a Vertex Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) para obtener más información sobre Vertex Pipelines.

Este bloc de notas se diseñó para ejecutarse en [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) o en [AI Platform Notebooks](https://cloud.google.com/ai-platform-notebooks). Si no usa ninguno de estos, simplemente puede hacer clic en el botón "Ejecutar en Google Colab" que se muestra más arriba.

## Preparación

Antes de ejecutar este bloc de notas, asegúrese de tener lo siguiente:

- Un proyecto de [Google Cloud Platform](http://cloud.google.com/).
- Un cubo de [Google Cloud Storage](https://cloud.google.com/storage). Consulte la [guía de creación de cubos](https://cloud.google.com/storage/docs/creating-buckets).
- Habilite [Vertex AI y la API de Cloud Storage](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,storage-component.googleapis.com).

Consulte la [documentación de Vertex](https://cloud.google.com/vertex-ai/docs/pipelines/configure-project) para configurar su proyecto de GCP con más detalle.

### Instale paquetes de Python

Instalaremos los paquetes de Python necesarios, incluidos TFX y KFP, para crear canalizaciones de aprendizaje automático y enviar trabajos a canalizaciones de Vertex.

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"

### Desinstalación de shapely

TODO(b/263441833) Esta es una solución temporal para evitar un ImportError. En última instancia, debería solucionarse admitiendo una versión reciente de Bigquery, en lugar de desinstalar otras dependencias adicionales.

In [ ]:
!pip uninstall shapely -y

#### ¿Reinició el tiempo de ejecución?

Si está usando Google Colab, la primera vez que ejecute la celda anterior, debe hacer clic en el botón "REINICIAR TIEMPO DE EJECUCIÓN" o usar el menú "Tiempo de ejecución &gt; Reiniciar tiempo de ejecución ..." para reiniciar el tiempo de ejecución. Esto se debe a la forma en que Colab carga los paquetes.

Si no está en Colab, puede reiniciar el tiempo de ejecución con la siguiente celda.

In [ ]:
# docs_infra: no_execute
import sys
if not 'google.colab' in sys.modules:
  # Automatically restart kernel after installs
  import IPython
  app = IPython.Application.instance()
  app.kernel.do_shutdown(True)

### Inicie sesión en Google para este bloc de notas

Si está ejecutando este bloc de notas en Colab, autentíquese con su cuenta de usuario:

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

**Si está en AI Platform Notebooks**, autentíquese con Google Cloud antes de ejecutar la siguiente sección, ejecutando lo siguiente

```sh
gcloud auth login
```

**en la ventana Terminal** (que puede abrir a través de **Archivo** &gt; **Nuevo** en el menú). Solo tiene que hacer esto una vez por instancia de bloc de notas.

Verifique las versiones del paquete.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

### Configuración de variables

Configuraremos algunas variables que se usan para personalizar las canalizaciones a continuación. Se requiere la siguiente información:

- Id del proyecto GCP. Consulte [Cómo identificar su id de proyecto](https://cloud.google.com/resource-manager/docs/creating-managing-projects#identifying_projects).
- Región de GCP para ejecutar canalizaciones. Para obtener más información sobre las regiones en las que las canalizaciones de Vertex están disponible, consulte la [guía de ubicaciones de Vertex AI](https://cloud.google.com/vertex-ai/docs/general/locations#feature-availability).
- Cubo de Google Cloud Storage para almacenar resultados de canalización.

**Ingrese los valores requeridos en la celda a continuación antes de ejecutarlo**.


In [ ]:
GOOGLE_CLOUD_PROJECT = ''     # <--- ENTER THIS
GOOGLE_CLOUD_REGION = ''      # <--- ENTER THIS
GCS_BUCKET_NAME = ''          # <--- ENTER THIS

if not (GOOGLE_CLOUD_PROJECT and GOOGLE_CLOUD_REGION and GCS_BUCKET_NAME):
    from absl import logging
    logging.error('Please set all required parameters.')

Configure `gcloud` para usar su proyecto.

In [ ]:
!gcloud config set project {GOOGLE_CLOUD_PROJECT}

In [ ]:
PIPELINE_NAME = 'penguin-vertex-pipelines'

# Path to various pipeline artifact.
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module.
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for input data.
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving.
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))

### Cómo preparar datos de ejemplo

Usaremos el mismo [conjunto de datos Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) que el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

Hay cuatro características numéricas en este conjunto de datos que ya fueron normalizadas para tener un rango [0,1]. Compilaremos un modelo de clasificación que prediga las `species` de pingüinos.

Tenemos que hacer nuestra propia copia del conjunto de datos. Debido a que TFX ExampleGen lee entradas de un directorio, tenemos que crear un directorio y copiar el conjunto de datos en él.

In [ ]:
!gsutil cp gs://download.tensorflow.org/data/palmer_penguins/penguins_processed.csv {DATA_ROOT}/

Veamos rápidamente al archivo CSV.

In [ ]:
!gsutil cat {DATA_ROOT}/penguins_processed.csv | head

## Cómo crear una canalización

Las canalizaciones de TFX se definen mediante las API de Python. Definiremos una canalización que consta de tres componentes, CsvExampleGen, Trainer y Pusher. La definición del modelo y la canalización es casi la misma que la del [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

La única diferencia es que no es necesario que configuremos `metadata_connection_config` que se utiliza para ubicar la base de datos de [ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd). Como Vertex Pipelines usa un servicio de metadatos administrado, los usuarios no deben preocuparse por él y nosotros no tenemos necesidad de especificar el parámetro.

Antes de definir realmente la canalización, debemos escribir un código modelo para el componente Trainer.

### Escriba el código del modelo

Usaremos el mismo código de modelo que en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils


from tfx import v1 as tfx
from tfx_bsl.public import tfxio

from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _make_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Copie el archivo del módulo en GCS, al que se puede acceder desde los componentes de la canalización. Como el entrenamiento del modelo se lleva a cabo en GCP, debemos cargar esta definición de modelo.

De lo contrario, es posible que sea conveniente compilar una imagen de contenedor que incluya el archivo del módulo y usar la imagen para ejecutar la canalización.

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/

### Cómo escribir una definición de canalización

Definiremos una función para crear una canalización de TFX.

In [ ]:
# Copied from https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple and
# slightly modified because we don't need `metadata_path` argument.

def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     ) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=components)

## Ejecute la canalización en canalizaciones de Vertex

Usamos `LocalDagRunner` que se ejecuta en un entorno local en el [Tutorial de canalizaciones simples de TFX](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple). TFX ofrece múltiples orquestadores para ejecutar su canalización. En este tutorial usaremos Vertex Pipelines junto con el ejecutor de DAG de Kubeflow V2.

Tenemos que definir un ejecutor para ejecutar realmente la canalización. Compilará su canalización en nuestro formato de definición de canalización con ayuda de las API de TFX.

In [ ]:
# docs_infra: no_execute
import os

PIPELINE_DEFINITION_FILE = PIPELINE_NAME + '_pipeline.json'

runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
    config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
    output_filename=PIPELINE_DEFINITION_FILE)
# Following function will write the pipeline definition to PIPELINE_DEFINITION_FILE.
_ = runner.run(
    _create_pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=PIPELINE_ROOT,
        data_root=DATA_ROOT,
        module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
        serving_model_dir=SERVING_MODEL_DIR))

El archivo de definición generado se puede enviar mediante el cliente kfp.

In [ ]:
# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs
import logging
logging.getLogger().setLevel(logging.INFO)

aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location=GOOGLE_CLOUD_REGION)

job = pipeline_jobs.PipelineJob(template_path=PIPELINE_DEFINITION_FILE,
                                display_name=PIPELINE_NAME)
job.submit()

Ahora puede visitar el enlace en el resultado anterior o visitar 'Vertex AI &gt; Canalizaciones' en [Google Cloud Console](https://console.cloud.google.com/) para ver el progreso.